In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal

import os
from tqdm import tqdm

path1 = 'C:/Users/bachu/Documents/ISRO-20200828T092236Z-001/ISRO/corpus/ArgentinaChangePair/Argentina_01131994.jpg'
path2 = 'C:/Users/bachu/Documents/ISRO-20200828T092236Z-001/ISRO/corpus/ArgentinaChangePair/Argentina_01202014.jpg'

# path1 = input('Enter path to image 1')
# path2 = input('Enter path to image 2')

In [2]:
def show_images(imgs, titles):
    for i in range(len(imgs)):
        cv2.imshow(titles[i], imgs[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
class GMM:
    def __init__(self, components, iterations = 5):
        self.components = components
        self.iterations = iterations
        self.count = 0

    def init(self, img):
        self.shape = img.shape
        self.n, self.m = self.shape

        self.phi = np.full(shape = self.components, fill_value = 1/self.components)
        self.weights = np.full(shape = self.shape, fill_value = 1/self.components)
        
        rand_r = np.random.randint(low = 0, high = self.n, size = self.components)
        self.mu = [img[index,:] for index in rand_r]
        self.sigma = [np.cov(img.T) for _ in range(self.components)]

    def e_step(self, img):
        self.weights = self.predict_probabilities(img)
        self.phi = self.weights.mean(axis=0)
    
    def m_step(self, img):
        for i in tqdm(range(self.components)):
            weight = self.weights[:, [i]]
            sum_weight = weight.sum()
            self.mu[i] = (img * weight).sum(axis=0) / sum_weight
            self.sigma[i] = np.cov(img.T, aweights=(weight/sum_weight).flatten(), bias=True)

    def fit(self, img):
        self.init(img)
        
        for _ in range(self.iterations):
#             print('Iteration {}'.format(self.count))
            self.e_step(img)
            self.m_step(img)
            self.count += 1
            
    def predict_probabilities(self, img):
        likelihood = np.zeros( (self.n, self.components) )
        for i in range(self.components):
            distribution = multivariate_normal(mean=self.mu[i], cov=self.sigma[i]) 
            likelihood[:, i] = distribution.pdf(img)
        
        num = likelihood * self.phi
        den = num.sum(axis=1)[:, np.newaxis]
        weights = num / den
        return weights
    
    def predictions(self, img):
        weights = self.predict_probabilities(img)
        return np.argmax(weights, axis=1)
    
    def get_mean_variance(self):
        return self.mu, self.sigma

In [4]:
X1 = cv2.imread(path1, 0)
X2 = cv2.imread(path2, 0)

factor = int(input('Enter zoom-out factor'))

length, width = X1.shape[:2]
print(length, width)

X1 = cv2.resize(X1, (int(width/factor), int(length/factor)))
X2 = cv2.resize(X2, (int(width/factor), int(length/factor)))

show_images([X1, X2], ['Image 1', 'Image 2'])

Enter zoom-out factor5
1200 2133


In [8]:
X_diff = cv2.absdiff(X1, X2)

show_images([X_diff], ['Difference Image'])

cv2.imwrite('Image1.jpg', X1)
cv2.imwrite('Image2.jpg', X2)
cv2.imwrite('Difference.jpg', X_diff)


True

In [6]:
gmm_model = GMM(components = 2, iterations = 5)
X_preproc = X_diff
gmm_model.fit(X_preproc.reshape(X_preproc.size, 1))

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 286.54it/s]


In [7]:
means = gmm_model.get_mean_variance()[0]
threshold = np.mean(means)
X_res = X_diff > threshold
X_res = X_res.astype(np.uint8) * 255.
show_images([X_res], ['GMM Segmentation'])

In [9]:
cv2.imwrite('Change_Map.jpg', X_res)

True